In [9]:
from base_code import *

valid_data = pd.read_csv('../neurosurgeon35_data/neurosurgeon35.csv', index_col='Respondent ID')
xai25 = pd.read_csv('../computational_data/xai25.csv')
mri_wise = pd.read_csv('../computational_data/mri_wise_acc.csv')
dr_wise = pd.read_csv('../computational_data/dr_wise_acc.csv')

In [10]:
dr_wise

,Respondent ID,DR,DR+AI,DR+XAI,MRI_DR,MRI_DR+AI,MRI_DR+XAI,XAI Qual,XAI Qual Std,Need XAI,Position,Year of Med,Year of Neurosurgery,AI Familiarity,AI Attitude
0,1,0.680000,0.680000,0.720000,25,25,25,5.240000,1.241934,0.333333,Attending,23.0,13.0,hear of AI,Skeptical
1,2,0.666667,1.000000,1.000000,3,2,2,5.000000,0.000000,1.000000,Attending,NaN,NaN,NaN,NaN
2,3,0.840000,0.840000,0.840000,25,25,25,1.040000,0.773563,0.160000,Attending,33.0,23.0,use AI in work/life,Not interested
3,4,0.800000,0.920000,0.920000,25,25,25,8.680000,2.185772,1.000000,Resident,1.0,1.0,use AI in work/life,Interested
4,5,0.880000,0.920000,0.920000,25,25,25,7.833333,1.027402,0.086957,Resident,3.0,3.0,hear of AI,Interested
5,6,0.760000,0.840000,0.880000,25,25,25,5.250000,1.534329,0.120000,Attending,14.0,4.0,hear of AI,Skeptical
6,7,0.840000,0.800000,0.800000,25,25,25,6.208333,2.235680,0.960000,Resident,4.0,4.0,hear of AI,Neutral
7,8,0.840000,0.960000,0.920000,25,25,25,5.040000,2.068429,0.280000,Resident,7.0,7.0,hear of AI,Skeptical
8,9,0.875000,0.875000,1.000000,8,8,8,7.250000,2.165064,0.250000,Resident,NaN,NaN,NaN,NaN
9,10,0.800000,0.840000,0.840000,25,25,25,6.600000,2.607681,0.200000,Attending,13.0,6.0,NaN,Excited


In [14]:
# reporting table in Supplemental S2
# Row (conditions): AI, dr, dr + AI, dr+ XAI, 
# Col (perf metrics): acc, sensitivity (recall_score), specificity, F1, MCC, auc  
# Note that in binary classification, recall of the positive class is also known as “sensitivity”; recall of the negative class is “specificity”.
from scipy.stats import friedmanchisquare, wilcoxon


metric_dict = {'Acc.': (accuracy_score, None), 'F1 grade II/III': (f1_score, 0), 'F1 GBM': (f1_score, 1), 'MCC': (matthews_corrcoef, None), 
              'Sen. grade II/III/Spec. GBM': (recall_score, 0), 'Sen. GBM/Spec. grade II/III': (recall_score, 1),
              "PPV grade II/III": (precision_score, 0), "PPV GBM": (precision_score, 1)}

def print_perf(k):
    metric, pos_label = metric_dict[k][0],  metric_dict[k][1]
#     print(k, metric, pos_label)
    if pos_label is not None:
        ai_alone = metric(mri_wise['gt'], mri_wise['pred'], pos_label = pos_label)
    else:
        ai_alone = metric(mri_wise['gt'], mri_wise['pred'])
    id_dfs, num_mri = get_drwise_perf_metric(valid_data, xai25, perf_metric = metric, pos_label = pos_label, complete_response_only=False)
    # id_dfs
    dr, ai, xai  =  id_dfs['dr'], id_dfs['AI'], id_dfs['XAI']
    stat_friedman, p_friedman = friedmanchisquare(*[dr, ai, xai])
    w, p1 = wilcoxon(np.array(ai), np.array(dr))#, alternative  = 'greater')
#     print(w, p1*3)
    w, p2 = wilcoxon(np.array(xai), np.array(dr))#, alternative  = 'greater')
#     print(w, p2*3)
    w, p3 = wilcoxon(np.array(xai) ,np.array(ai))# alternative  = 'greater')
#     print(w, p3*3)
    output = '\\textbf{{{}}} & {:.4f} & ${:.4f}\pm{:.4f}$ &${:.4f}\pm{:.4f}$ &${:.4f}\pm{:.4f}$ & {:.4g} & {:.4g}& {:.4g}& {:.4g} \\\\ \\hline'.format(k, ai_alone, dr.mean(), dr.std(), ai.mean(), ai.std(), xai.mean(), xai.std(), p_friedman, p1*3, p2*3, p3*3)
    return output

output = ''
for k in metric_dict:
    output += print_perf(k)
    output += '\n'
print(output)

/Users/root202/Software/anaconda3/envs/posthoc-eval/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/root202/Software/anaconda3/envs/posthoc-eval/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/root202/Software/anaconda3/envs/posthoc-eval/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

\textbf{Acc.} & 0.8800 & $0.8249\pm0.0869$ &$0.8770\pm0.0733$ &$0.8852\pm0.0702$ & 7.755e-06 & 0.001543& 0.0004037& 0.3454 \\ \hline
\textbf{F1 grade II/III} & 0.8000 & $0.6424\pm0.2257$ &$0.6997\pm0.2356$ &$0.7116\pm0.2390$ & 4.341e-05 & 0.002134& 0.0006973& 0.5456 \\ \hline
\textbf{F1 GBM} & 0.9143 & $0.8777\pm0.0644$ &$0.9126\pm0.0559$ &$0.9187\pm0.0535$ & 7.755e-06 & 0.002895& 0.0008773& 0.6383 \\ \hline
\textbf{MCC} & 0.7181 & $0.5448\pm0.2274$ &$0.6242\pm0.2340$ &$0.6415\pm0.2365$ & 4.341e-05 & 0.002134& 0.0006985& 0.5456 \\ \hline
\textbf{Sen. grade II/III/Spec. GBM} & 0.8571 & $0.6694\pm0.2444$ &$0.7184\pm0.2510$ &$0.7224\pm0.2521$ & 0.0001117 & 0.0178& 0.01081& 0.9519 \\ \hline
\textbf{Sen. GBM/Spec. grade II/III} & 0.8889 & $0.8800\pm0.0961$ &$0.9060\pm0.0870$ &$0.9156\pm0.0834$ & 0.001765 & 0.05405& 0.008309& 0.7735 \\ \hline
\textbf{PPV grade II/III} & 0.7500 & $0.6412\pm0.2502$ &$0.7041\pm0.2608$ &$0.7226\pm0.2635$ & 4.341e-05 & 0.006771& 0.001361& 0.7418 \\ \hline
\textbf